# Traditional Approach Demo Code

This notebook will contain the code necessary to generate the predictions file using preprocessing techniques and model generated from the `develop_deep.ipynb` notebook.

For this, a deep learning model is used with DINOv2 as the model used for feature extraction for image preprocessing

## Miscellaneous

In [ ]:
# %pip install tensorflow transformers torch torchvision torchaudio scikit-learn opencv-python numpy pickle5 tqdm -q

# To hide warnings produced by different packages
import warnings
warnings.filterwarnings('ignore')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.9 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not 

## Imports

In [ ]:
# from transformers import AutoImageProcessor, Dinov2Model
import torch
from torch import nn

from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

from copy import deepcopy
import typing
import os
from pathlib import Path
import numpy as np
from PIL import Image

import cv2
import cv2.typing as cv_typing
from google.colab import drive
from tensorflow.keras.preprocessing import image
import csv
# drive.mount('/content/drive')

Mounted at /content/drive


## Constants

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

classes = ["Bacteria", "Fungi", "Healthy", "Pest", "Phytopthora", "Virus"]

# Path of where the train images are located
# img_dir = "/content/drive/MyDrive/DCS/CS180/Project/potato_test"
img_dir = "../data/potato_test" # if local

# Path for final model
# model_dir = "/content/drive/MyDrive/DCS/CS180/Project/models"
model_dir = "../models" # if local

# Other constants
ORIG_IMG_SIZE = (1500,1500)
BATCH_SIZE = 8
seed_value = 42
RESIZE_IMG = (420, 420)

# Set seed
torch.manual_seed(seed_value)

In [ ]:
device

device(type='cuda')

In [ ]:
# Load the vits14 version of dinov2
dino_model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
dino_model

Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vits14/dinov2_vits14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vits14_pretrain.pth
100%|██████████| 84.2M/84.2M [00:00<00:00, 162MB/s]


DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-11): 12 x NestedTensorBlock(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (ls2): LayerScale()
      (drop_path2): Identity()
    )
  )
  (n

In [ ]:
class DinoVisionTransformerClassifier(nn.Module):
    def __init__(self):
        super(DinoVisionTransformerClassifier, self).__init__()
        self.transformer = deepcopy(dino_model)
        # self.classifier = nn.Sequential(nn.Linear(384, 384), nn.ReLU(), nn.Linear(384, 1))
        self.classifier = nn.Sequential(nn.Dropout(0.5), nn.ReLU(), nn.Linear(in_features=384, out_features=len(classes), bias=True))
        # self.classifier = nn.Linear(in_features=384, out_features=len(classes), bias=True)

    def forward(self, x):
        x = self.transformer(x)
        x = self.transformer.norm(x)
        x = self.classifier(x)
        return x

In [ ]:
# Load the model
with open(Path(f"{model_dir}/model-3.pth"), 'rb') as file:
    torch.serialization.add_safe_globals([DinoVisionTransformerClassifier])
    loaded_model = torch.load(f"{model_dir}/model-3.pth", map_location=device, weights_only=False)
    loaded_model = loaded_model.to(device)
    loaded_model.eval()

## Functions

### Loading Data

`load_images` takes the directory where the test images are located and loads them into a program as a list

In [ ]:
def load_images(
    file_path: str = "./potato_test",
    resize_dim: tuple[int, int] = (518, 518),
) -> list[cv_typing.MatLike]:
    # Get folder
    dir = Path(file_path)

    # Check if directory
    if not dir.is_dir():
        raise Exception("Please enter a valid directory")

    # Get all images in the dir
    imgs = [os.path.join(dir, f) for f in os.listdir(dir) if os.path.isfile(os.path.join(dir, f))]

    # Variable for final array
    final_imgs: list[cv_typing.MatLike] = []
    final_filenames: list[str] = []

    try:
        for img_path in imgs:
            img_loaded: Image.Image = image.load_img(img_path, target_size=ORIG_IMG_SIZE)
            img_array: np.ndarray[typing.Any, typing.Any] = image.img_to_array(img_loaded)
            img_array = cv2.resize(img_array, resize_dim)
            final_imgs.append(img_array)

            filename = Path(img_path).name
            final_filenames.append(filename)
    except Exception as e:
        print(f"Failed to load images: {e}")

    return final_imgs, final_filenames

### Preprocessing through Feature Extraction

The `preprocess_img` takes a list of images and preprocesses them using DINOv2 by taking the features of the image.

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize(RESIZE_IMG),
    transforms.RandomHorizontalFlip(p=0.5), # Random flip with 50% probability
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.4846, 0.5446, 0.3977], std=[0.2117, 0.1950, 0.2329]),
])

class PotatoLeafDisease(Dataset):
    def __init__(self, imgs: list[np.ndarray], transforms: transforms.Compose):
        self.imgs = imgs
        self.transforms = transforms

    def __len__(self) -> int:
        return len(self.imgs)

    def __getitem__(self, idx: int) -> torch.Tensor:
        img = self.imgs[idx]

        # Convert numpy image to PIL Image
        image = Image.fromarray(img.astype(np.uint8))

        # Apply transform
        if self.transforms:
            image = self.transforms(image)

        return image

In [ ]:
def preprocess_img(
    imgs: list[np.ndarray[typing.Any, typing.Any]] = [],
) -> DataLoader:
    all_features = []

    leaves_data = PotatoLeafDisease(imgs, transforms=train_transform)
    return DataLoader(leaves_data, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# # Load the vits14 version of dinov2
# dino_model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
# dino_model

### Get Class Labels

`get_labels` simply converts the numerical labeling produced by the model into the actual class label names (e.g. "Healthy" instead of 2)

In [ ]:
def get_labels(
    y: np.ndarray[typing.Any, typing.Any],
    classes: list[str] = ["Bacteria", "Fungi", "Healthy", "Pest", "Phytopthora", "Virus"],
) -> np.ndarray[typing.Any, typing.Any]:
    fxn = lambda x: classes[x]
    applyall = np.vectorize(fxn)
    return applyall(y)

## Running

In [ ]:
# Load the images to predict
imgs_to_pred, filenames = load_images(img_dir)

# Preprocess images using DINOv2
dataloader = preprocess_img(imgs_to_pred)

# Make predictions
all_preds = []
with torch.no_grad():
    for batch in dataloader:
        batch = batch.to(device)
        preds = loaded_model(batch)
        predicted_classes = torch.argmax(preds, dim=1)
        all_preds.extend(predicted_classes.cpu().numpy())

# Turn into actual labels
final_labels = get_labels(all_preds)

# Save as text file
# with open(Path("/content/drive/MyDrive/DCS/CS180/Project/predictions/pred_trad.csv"), "w") as file:
with open('../predictions/pred_trad.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["image_filename", "predicted_label"])
    writer.writerows(zip(filenames, final_labels))